# Inner Functions


## Encapsulation

* Use inner functions to protect them from other parts of the program from extensive modification if the design decision is changed
* Known as **information hiding** or **encapsulation** 
* Following program returns an error if we call the inner function:

In [1]:
def outer(num1):
    def inner_increment(num1):  # Hidden from outer code
        return num1 + 1
    num2 = inner_increment(num1)
    print(num1, num2)

inner_increment(10)

NameError: name 'inner_increment' is not defined

In [2]:
outer(10) # no error

10 11


* Recursive Example:

In [3]:
def factorial(number):

    # Error handling
    if not isinstance(number, int):
        raise TypeError("Sorry. 'number' must be an integer.")
    if not number >= 0:
        raise ValueError("Sorry. 'number' must be zero or positive.")
    
    # Nested function
    def inner_factorial(number):
        if number <= 1:
            return 1
        return number*inner_factorial(number-1)
    return inner_factorial(number)

# Call the outer function.
print(factorial(4))

24


**Advantage of above implementation:**
* Perform all argument checking in outer function
* Safely skip error checking altogether in the inner function

## Keepin' it DRY

* DRY = Don't repeat yourself
* principle of software development aimed at reducing repetition of software patterns, replacing it with abstractions or using data normalization to avoid redundancy
* "Every piece of knowledge must have a single, unambiguous, authoritative representation within a system"
* E.g. want to know number of wi-fi hotspots in New York:

In [4]:
import pandas as pd

new_york_wifi = pd.read_csv('data/NYC_Wi-Fi_Hotspot_Locations.csv')
new_york_wifi.sample(5)

,OBJECTID,Borough,Type,Provider,Name,Location,Latitude,Longitude,X,Y,...,Neighborhood Tabulation Area (NTA),Council Distrcit,Postcode,BoroCD,Census Tract,BCTCB2010,BIN,BBL,DOITT_ID,"Location (Lat, Long)"
700,10053,3,Free,BPL,Leonard - Brooklyn Public Library,81 DEVOE STREET,40.713645,-73.947936,9.986835e+05,199280.871173,...,East Williamsburg,34,11211,301,503,503,3068818,3027620021,475,"(40.713644888, -73.9479357128)"
1240,12640,3,Free,LinkNYC - Citybridge,bk-02-126807,56 LAFAYETTE AVENUE,40.686961,-73.976371,9.908032e+05,189555.581241,...,Fort Greene,35,11217,302,35,35,3059247,3021130020,3640,"(40.6869605882, -73.9763707911)"
344,12869,1,Free,LinkNYC - Citybridge,mn-08-119830,1675 3 AVENUE,40.783289,-73.950562,9.979411e+05,224653.951004,...,Yorkville,5,10128,108,154,154,1048916,1015397500,3373,"(40.7832887698, -73.9505618001)"
517,12506,2,Free,LinkNYC - Citybridge,bx-04-119048,105 EAST 165 STREET,40.831467,-73.921747,1.005905e+06,242212.795152,...,West Concourse,16,10452,204,195,195,2002953,2024780060,3591,"(40.8314669998, -73.9217469998)"
2658,11443,1,Free,LinkNYC - Citybridge,mn-05-137863,458 7 AVE,40.751540,-73.990365,9.869194e+05,213082.967257,...,Midtown-Midtown South,3,10001,105,109,109,1014409,1007840050,3939,"(40.7515396216, -73.9903653318)"


In [5]:
def process(file_name):

    def do_stuff(file_process):
        wifi_locations = {}

        for line in file_process:
            values = line.split(',')
            # Build the dict and increment values.
            try:
                wifi_locations[values[1]] = wifi_locations.get(values[1], 0) + 1
            except:
                continue

        max_key = 0
        for name, key in wifi_locations.items():
            if key > max_key:
                max_key = key
                business = name
        all_locations = sum(wifi_locations.values())

        print(f'There are {all_locations} WiFi hotspots in NYC, '
              f'and {business} has the most with {max_key}.')

    if isinstance(file_name, str):
        with open(file_name, 'r') as f:
            do_stuff(f)
    else:
        do_stuff(file_name)

In [6]:
process('data/NYC_Wi-Fi_Hotspot_Locations.csv')

There are 3321 WiFi hotspots in NYC, and 1 has the most with 1672.


## Closures and Factory Functions

* A Closure is a function object that remembers values in enclosing scopes even if they are not present in memory.
* It is a record that stores a function together with an environment: 
    - a mapping associating each free variable of the function (variables that are used locally, but defined in an enclosing scope) 
    - with the value or reference to which the name was bound when the closure was created.
* A closure—unlike a plain function—allows the function to access those captured variables through the closure’s copies of their values or references, even when the function is invoked outside their scope.
* Beginners often think that a closure is the inner function, but it’s really caused by the inner function.
* The closure “closes” the local variable on the stack, and this stays around after the stack creation has finished executing.

In [7]:
def generate_power(number):
    # Define the inner function ...
    def nth_power(power):
        return number ** power
    # ... that is returned by the factory function.

    return nth_power

In [8]:
raise_two = generate_power(2)
raise_three = generate_power(3)
print(raise_two(7))
print(raise_three(5))

128
243


### What’s Happening in the Example

1. `generate_power()` is a factory function, which simply means that it creates a new function each time it is called and then returns the newly created function. Thus, `raise_two()` and `raise_three()` are the newly created functions.
2. new inner function - takes a single argument, `power`, and returns `number**power`.
3. `nth_power()` gets the value of `power` from the outer function, the factory function:
    - Call the outer function: `generate_power(2)`
    - Build `nth_power()`, which takes a single argument `power`
    - Take a snapshot of the state of `nth_power()`, which includes `number=2`
    - Pass that snapshot into `generate_power()`
    - Return `nth_power()`

**In summary**
* the closure “initializes” the number bar in `nth_power()` and then returns it.
* whenever you call that newly returned function, it will always see its own private snapshot that includes `number=2`

In [9]:
def has_permission(page):
    def inner(username):
        if username == 'Admin':
            return "'{0}' does have access to {1}.".format(username, page)
        else:
            return "'{0}' does NOT have access to {1}.".format(username, page)
    return inner


current_user = has_permission('Admin Area')
print(current_user('Admin'))

random_user = has_permission('Admin Area')
print(random_user('Not Admin'))

'Admin' does have access to Admin Area.
'Not Admin' does NOT have access to Admin Area.


* simplified function to check if a certain user has the correct permissions to access a certain page.
* could easily modify this to grab the user in session to check if they have the correct credentials to access a certain route
* instead of checking if the user is just equal to 'Admin', could query the database to check the permission and then return the correct view depending on whether the credentials are correct or not.

## When and why to use Closures:

* As closures are used as callback functions, they provide some sort of data hiding. This helps us to reduce the use of global variables.
* When we have few functions in our code, closures prove to be efficient way. But if we need to have many functions, then go for class (OOP).

## References

1. [Wikipedia - Information Hiding](https://en.wikipedia.org/wiki/Information_hiding)
2. [Wikipedia - Don't repeat yourself](https://en.wikipedia.org/wiki/Don't_repeat_yourself)
3. [RealPython - Inner functions](https://realpython.com/inner-functions-what-are-they-good-for/)
4. [GFG - Python Closure](https://www.geeksforgeeks.org/python-closures/)